In [0]:
import mlflow.pyfunc
import pandas as pd
from sklearn.preprocessing import StandardScaler

class SuperFE(mlflow.pyfunc.PythonModel):
    
    def __init__(self, features_to_scale: list, pk_col: str, target_col: str):
        self.features_to_scale = features_to_scale
        self.pk_col = pk_col
        self.target_col = target_col
        self.scaler = None
        self.is_fitted = False

    def fit(self, X: pd.DataFrame):
        self.scaler = StandardScaler()
        cols_presentes = [c for c in self.features_to_scale if c in X.columns]
        self.scaler.fit(X[cols_presentes])
        self.is_fitted = True
        return self

    def predict(self, context, model_input: pd.DataFrame) -> pd.DataFrame:
        if not self.is_fitted:
            raise Exception("SuperFE debe ejecutarse con fit() antes de predict().")
        
        X_copy = model_input.copy()
        cols_presentes = [c for c in self.features_to_scale if c in X_copy.columns]
        X_copy[cols_presentes] = self.scaler.transform(X_copy[cols_presentes])

        cols_finales = [self.pk_col] + cols_presentes + [self.target_col]
        cols_finales = [c for c in cols_finales if c in X_copy.columns]
        
        return X_copy[cols_finales]